# Get the Exact Answer
Start off by computing the exact Hessian to use a reference point. 
First relax the structure then compute the Hessians using [ase's Vibrations module](https://databases.fysik.dtu.dk/ase/ase/vibrations/modes.html), which will compute them numerically using central derivatives

In [1]:
from ase.thermochemistry import IdealGasThermo
from ase.vibrations import VibrationsData, Vibrations
from ase.calculators.psi4 import Psi4
from ase.optimize import QuasiNewton
from ase.io import write
from ase import Atoms
from pathlib import Path
import numpy as np
import shutil
import json
import os

Configuration

In [2]:
molecule_name = 'caffeine'
method = 'hf'
basis = 'def2-svpd'
threads = min(os.cpu_count(), 12)

Derived

In [3]:
run_name = f'{molecule_name}_{method}_{basis}'

## Load in Target Molecule
We have it in a JSON file from PubChem

In [4]:
def load_molecule(name: str) -> Atoms:
    """Load a molecule from a PubChem JSON file
    
    Args:
        name: Name of the molecule
    Returns:
        ASE Atoms object
    """
    
    # Get the compound data
    with open(f'data/structures/{name}.json') as fp:
        data = json.load(fp)
    data = data['PC_Compounds'][0]
        
    # Extract data from the JSON
    atomic_numbers = data['atoms']['element']
    positions = np.zeros((len(atomic_numbers), 3))
    conf_data = data['coords'][0]['conformers'][0]
    for i, c in enumerate('xyz'):
        if c in conf_data:
            positions[:, i] = conf_data[c]
        
    # Build the object    
    return Atoms(numbers=atomic_numbers, positions=positions)

In [5]:
atoms = load_molecule(molecule_name)

## Perform the Geometry Optimization
Build the ASE calculator then run QuasiNewton to a high tolerance

In [6]:
calc = Psi4(method=method, basis=basis, num_threads=threads, memory='4096MB')


  Memory set to   3.815 GiB by Python driver.
  Threads set to 12 by Python driver.


In [7]:
%%time
atoms.calc = calc
dyn = QuasiNewton(atoms)
dyn.run(fmax=0.01)


  Memory set to   3.815 GiB by Python driver.
  Threads set to 12 by Python driver.
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 08:51:24   -18390.794139        4.1661
BFGSLineSearch:    1[  2] 08:51:57   -18391.020663        1.9419
BFGSLineSearch:    2[  4] 08:52:30   -18391.134457        1.3043
BFGSLineSearch:    3[  6] 08:53:02   -18391.190767        1.0587
BFGSLineSearch:    4[  8] 08:53:36   -18391.215384        0.5745
BFGSLineSearch:    5[ 10] 08:54:08   -18391.226242        0.3910
BFGSLineSearch:    6[ 12] 08:54:41   -18391.233269        0.3498
BFGSLineSearch:    7[ 14] 08:55:13   -18391.239126        0.2510
BFGSLineSearch:    8[ 16] 08:55:46   -18391.244272        0.2339
BFGSLineSearch:    9[ 18] 08:56:18   -18391.247255        0.1999
BFGSLineSearch:   10[ 20] 08:56:51   -18391.249084        0.1266
BFGSLineSearch:   11[ 22] 08:57:23   -18391.249937        0.1217
BFGSLineSearch:   12[ 24] 08:57:55   -18391.251073        0.1063
BFGSL

True

Save the output file

In [8]:
out_dir = Path('data') / 'exact'
out_dir.mkdir(exist_ok=True)

In [9]:
write(out_dir / f'{run_name}.xyz', atoms)

## Compute the Hessian
Use Psi4's analytic gradients to get a runtime, but ASE's for the actual value

In [10]:
if Path('vib').is_dir():
    shutil.rmtree('vib')

In [11]:
%%time
vib = Vibrations(atoms)
vib.run()

CPU times: user 4h 21min 47s, sys: 8min 58s, total: 4h 30min 46s
Wall time: 24min 47s


Save the vibration data

In [12]:
vib_data = vib.get_vibrations()
with (out_dir / f'{run_name}.json').open('w') as fp:
    vib_data.write(fp)